# Script to run LePhare on a matched table

## Requirements

This notebook is supposed to be run on a matched LePhare input file after carrying out the matching using the `match_tables.ipynb` notebook.

## Structure

- Load the region and the relevant matched and processd files
- Running `LePhare`:
  1. Collect the filter information
  2. Collect the template information
     1. For the stellar library
     2. For the pointlike library
     3. For the extended library
  3. Run the photo-z-routine

In [1]:
import os
import function_package as fp
import logging
logging.getLogger().setLevel("INFO")

logging.info("Modules loaded successfully")
# Ignore the 'Could not import regions' line, it is from one of the submodules

INFO:root:Modules loaded successfully


Could not import regions, which is required for some of the functionalities of this module.


In [2]:
# Load the region that has been saved before. If this fails, make sure to run `match_tables.ipynb` first!
REGION = fp.Region(load_from_disk=True)
logging.info(REGION)

INFO:root:Region constrained to 125.0000 <= RA <= 130.0000 and -5.0000 <= DEC <= 0.0000.
This corresponds to a linear (!) size of 25 deg^2


## Run LePhare

Before we run any of the subprograms, let us initialise important stems that we're going to use over and over:

In [3]:
LEPHAREDIR = fp.get_lephare_directory("dir")
LEPHAREWORK = fp.get_lephare_directory("work")
PARAFILE = fp.get_filepath("para_in")
PARA_OUT = fp.get_filepath("para_out")

FILTER_STEM = "ls10plus"

STAR_STEM = "base"
STAR_FILE = fp.get_filepath("template", "star", stem=STAR_STEM)
STAR_SEDLIB_NAME = f"{STAR_STEM}_star_sedlib"
STAR_MAGLIB_NAME = f"{STAR_STEM}_star_maglib"
TEMP_STEM = "combined"  # This refers to the templates that seemed to work best in my thesis
PLIKE_FILE = fp.get_filepath("template", "pointlike", stem=TEMP_STEM)
PLIKE_SEDLIB_NAME = f"{TEMP_STEM}_pointlike_sedlib"
PLIKE_MAGLIB_NAME = f"{TEMP_STEM}_pointlike_maglib"
EXT_FILE = fp.get_filepath("template", "extended", stem=TEMP_STEM)
EXT_SEDLIB_NAME = f"{TEMP_STEM}_extended_sedlib"
EXT_MAGLIB_NAME = f"{TEMP_STEM}_extended_maglib"

RUN_STEM = "base"
PLIKE_IN = fp.get_filepath("lephare_in", "pointlike", stem=RUN_STEM)
PLIKE_OUT = fp.get_filepath("lephare_out", "pointlike", stem=RUN_STEM)
EXT_IN = fp.get_filepath("lephare_in", "extended", stem=RUN_STEM)
EXT_OUT = fp.get_filepath("lephare_out", "extended", stem=RUN_STEM)


### 1) Run the filter file

> **Notes**:\
> This step can be skipped if the filterfile has already been set up before.\
> If you wanted to change the bands used, you need to change some parameters in the parameter file as well!\
> Also, make sure to rerun the commands to produce the magnitude libraries in this case!

In [4]:
# We use this stem for the Galex, sweep (+WISE) and VHS filters:
# FUV, NUV, g, r, i, z, J, H, Y, Ks, W1, W2, W3, W4
FILTER_OUT = fp.get_filepath("filter", stem=FILTER_STEM)
FILTER_REPO = fp.get_lephare_directory("filter_origin")

In [16]:
!"$LEPHAREDIR"/source/filter -c "$PARAFILE" \
    -FILTER_REP "$FILTER_REPO" \
    -FILTER_FILE "$FILTER_STEM" \
    >"$FILTER_OUT"

terminate called after throwing an instance of 'std::invalid_argument'
  what():  ERROR: can't open config file /hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para
/bin/bash: line 1: 2755462 Aborted                 (core dumped) "/hs/fs14/data/4most/4hi-q/LEPHARE/"/source/filter -c "/hshome/bat3083/sel_4hi_q_v2/data/lephare/parameters/base_in.para" -FILTER_REP "/hshome/bat3083/sel_4hi_q_v2/data/lephare/filters/individual_files/" -FILTER_FILE "ls10plus" > FILTER_OUT


### 2) Run the template files

Here, we create the template files that LePhare uses to fit the SEDs against.\
The creation usually consists of two steps:
1. The creation of the SED library:\
   After we've provided a list of the filenames where our template files are located, LePhare unifies them to a single library file.\
   This is done via the `sedtolib` command.
2. The creation of the magnitude library:\
   Then, these library SEDs have to be convolved through each of our filters to create physical magnitudes we would observe.\
   These are then propagated in a redshift grid, and, if requested, an EBV-grid to create the magnitude libraries that are later used for the fits.\
   This is done via the `mag_gal` command.

> **Notes**:\
> These steps can be skipped if the required template files already exist.\
> Be sure to repeat this step if using a different set of filters!\
> If you're introducing new template lists, make sure they're included in the ``data/lephare/templates`` directory!

#### 2a) The stellar template library

In [ ]:
%%bash
# This is the sedtolib routine to create the general stellar library.
{$LEPHAREDIR}/source/sedtolib -c $PARAFILE \
    -t S \
    -STAR_SED $STAR_FILE \
    -STAR_LIB $STAR_SEDLIB_NAME

In [ ]:
%%bash
# This is the mag_gal routine to create the convolved stellar magnitude library

{$LEPHAREDIR}/source/mag_gal -c $PARAFILE \
    -t S \
    -STAR_LIB_IN $STAR_SEDLIB_NAME \
    -STAR_LIB_OUT $STAR_MAGLIB_NAME \
    -EM_LINES NO \
    -LIB_ASCII YES \
    -FILTER_FILE $FILTER_STEM

#### 2b) The pointlike library

In [ ]:
%%bash
# This is the sedtolib routine to create the general pointlike library.
{$LEPHAREDIR}/source/sedtolib -c $PARAFILE \
    -t G \
    -GAL_SED $PLIKE_FILE \
    -GAL_LIB $PLIKE_SEDLIB_NAME

In [ ]:
%%bash
# This is the mag_gal routine to create the convolved pointlike magnitude library
# The last three lines correspond to the prescribed extinction laws, the models they should be applied on
# and the EB_Vs to consider for each application.

{$LEPHAREDIR}/source/mag_gal -c $PARAFILE \
    -t G \
    -GAL_LIB_IN $PLIKE_SEDLIB_NAME \
    -GAL_LIB_OUT $PLIKE_MAGLIB_NAME \
    -EM_LINES NO \
    -LIB_ASCII YES \
    -FILTER_FILE $FILTER_STEM \
    -EXTINC_LAW SMC_prevot.dat,SB_calzetti.dat \
    -MOD_EXTINC 0,0,0,30 \
    -EB_V 0.,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4

#### 2c) The extended library

In [ ]:
%%bash
# This is the sedtolib routine to create the general extended library.
{$LEPHAREDIR}/source/sedtolib -c $PARAFILE \
    -t G \
    -GAL_SED $EXT_FILE \
    -GAL_LIB $EXT_SEDLIB_NAME

In [ ]:
%%bash
# This is the mag_gal routine to create the convolved extended magnitude library
# We don't use extinction laws here.

{$LEPHAREDIR}/source/mag_gal -c $PARAFILE \
    -t G \
    -GAL_LIB_IN $EXT_SEDLIB_NAME \
    -GAL_LIB_OUT $EXT_MAGLIB_NAME \
    -EM_LINES NO \
    -LIB_ASCII YES \
    -FILTER_FILE $FILTER_STEM

> **Note**:\
> If you want to analyse the template files, they can be relocated to the 'template' directory as follows:

In [ ]:
for filename in [STAR_MAGLIB_NAME, PLIKE_MAGLIB_NAME, EXT_MAGLIB_NAME]:
    lib_prefix = "star" if "star" in filename else "mag"
    filepath = fp.get_lephare_directory("work") + "lib_" + lib_prefix
    sourcepath = f"{filepath}/{filename}.dat"
    destpath = fp.get_lephare_directory("templates") + filename + ".dat"
    # Simply use the Linux routine to copy the files:
    if fp.ask_file_overwrite(destpath):
        os.system(f"cp {sourcepath} {destpath}")
logging.info("All files copied successfully.")

### 3) Run the photo-z routine

This is the core functionality of LePhare.\
Be sure you know what you're doing when modifying the parameters.

#### 3a) Run the photo-zs for the pointlike sources

In [ ]:
%%bash
# Perform the zphota routine:
{$LEPHAREDIR}/source/zphota -c $PARAFILE \
    -ZPHOTLIB $PLIKE_MAGLIB_NAME,$STAR_MAGLIB_NAME \
    -CAT_IN $PLIKE_IN \
    -CAT_OUT $PLIKE_OUT \
    -PARA_OUT $PARA_OUT \
    -LIB_ASCII YES \
    -MAG_REF 7 \
    -MAG_ABS -30,-20
# Additional sometimes useful parameters:
# -PDZ_OUT YES  -> Generate .spec files for each source
# -SPEC_OUT YES  -> Generate .spec files for each source
# -CAT_LINES 10, 10  -> Only consider the first 10 rows (highly recommended with .spec files)
# -GLB_CONTEXT context  -> Limit the SED-Fitting to a subset of the bands described via 'context'

#### 3b) Run the photo-zs for the extended sources

In [ ]:
%%bash
# Perform the zphota routine:
{$LEPHAREDIR}/source/zphota -c $PARAFILE \
    -ZPHOTLIB $EXT_MAGLIB_NAME,$STAR_MAGLIB_NAME \
    -CAT_IN $EXT_IN \
    -CAT_OUT $EXT_OUT \
    -PARA_OUT $PARA_OUT \
    -GLB_CONTEXT -1 \
    -LIB_ASCII YES \
    -MAG_REF 7 \
    -MAG_ABS -24,-8
# Additional sometimes useful parameters:
# -PDZ_OUT YES  -> Generate .spec files for each source
# -SPEC_OUT YES  -> Generate .spec files for each source
# -CAT_LINES 10, 10  -> Only consider the first 10 rows (highly recommended with .spec files)
# -GLB_CONTEXT context  -> Limit the SED-Fitting to a subset of the bands described via 'context'